In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [59]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [60]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [115]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [62]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Prepare ETL for the Staff Assignment data

In [137]:
#class df as a parent to staff assignment df
staffassignment_df = pd.read_sql("SELECT * FROM staffassignment", conn)
staffassignment_df.head(30)

,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,1,1,1,Teacher Assistant,2020-09-16,None
1,2,4,1,Teacher Assistant,2020-09-16,None
2,3,3,1,Instructor,2020-09-16,None


In [138]:
#rename columns to match salesforce
staffassignment_df.rename(columns={
    'Role':'Role__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c'
}, inplace=True)
staffassignment_df.head()

,ID_StaffAssignment,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c
0,1,1,1,Teacher Assistant,2020-09-16,None
1,2,4,1,Teacher Assistant,2020-09-16,None
2,3,3,1,Instructor,2020-09-16,None


In [139]:
#select relevant columns
staffassignment_df = staffassignment_df[['ID_Staff', 'ID_Class', 'Role__c', 'StartDate__c', 'EndDate__c']]
staffassignment_df.head()

,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c
0,1,1,Teacher Assistant,2020-09-16,None
1,4,1,Teacher Assistant,2020-09-16,None
2,3,1,Instructor,2020-09-16,None


In [140]:
#format date columns
staffassignment_df['StartDate__c'] = pd.to_datetime(staffassignment_df['StartDate__c']).dt.date
staffassignment_df['EndDate__c'] = None
staffassignment_df

,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c
0,1,1,Teacher Assistant,2020-09-16,None
1,4,1,Teacher Assistant,2020-09-16,None
2,3,1,Instructor,2020-09-16,None


# Create Class Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Class` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [64]:
#extract class object from salesforce
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Section__c, Start_Date__c, Name FROM Class__c")
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'Section__c': row['Section__c'],
        'Start_Date__c': row['Start_Date__c']
    }
    class_lookup_list.append(rec)
    
class_lookup_list

[{'ID_Class__c': 'a014x000008VFla',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': '2019-05-14'},
 {'ID_Class__c': 'a014x000008VFlV',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': '2018-05-15'},
 {'ID_Class__c': 'a014x000008VFlQ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': '2020-09-16'},
 {'ID_Class__c': 'a014x000008VFkd',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': '2019-05-14'}]

In [65]:
#convert to DF
class_lookup_list = pd.DataFrame(class_lookup_list)
class_lookup_list

,ID_Class__c,Section__c,Start_Date__c
0,a014x000008VFla,GWARL201905UIUX3,2019-05-14
1,a014x000008VFlV,GWDC201805DATA3,2018-05-15
2,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
3,a014x000008VFkd,GWARL201905WEB3,2019-05-14


### Query original 'Class' table from MySQL
Join with class_lookup_list on 'Section' column
then join with staff assignment on the original ID_class

In [77]:
query = '''
    SELECT *
    FROM 
        class 
'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [78]:
#leave out 'ID_Course' column since it isn't in the staffassignment object
class_data_df = class_data_df[['ID_Class', 'Section']]
class_data_df

,ID_Class,Section
0,1,GWU-ARL-DATA-PT-09-0
1,2,GWDC201805DATA3
2,3,GWARL201905WEB3
3,4,GWARL201905UIUX3


In [79]:
#rename columns
class_data_df.rename(columns={
    'Section':'Section__c'
}, inplace=True)

class_data_df

,ID_Class,Section__c
0,1,GWU-ARL-DATA-PT-09-0
1,2,GWDC201805DATA3
2,3,GWARL201905WEB3
3,4,GWARL201905UIUX3


### Join the class DataFrame with the Class object from SF
This join is necessary to get a DF with the original ID_Class and the SF 'name'/'ID_Class__c'

In [82]:
class_lookup_df = pd.merge(class_lookup_list, class_data_df, how='left')

class_lookup_df

,ID_Class__c,Section__c,Start_Date__c,ID_Class
0,a014x000008VFla,GWARL201905UIUX3,2019-05-14,4
1,a014x000008VFlV,GWDC201805DATA3,2018-05-15,2
2,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,1
3,a014x000008VFkd,GWARL201905WEB3,2019-05-14,3


# Prepare ETL for the `Staff` data

In [119]:
# Staff DF from AWS MySQL
staff_df = pd.read_sql("SELECT * FROM staff", conn)
staff_df

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [120]:
#rename columns to match salesforce
staff_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate':'BirthDate__c'
}, inplace=True)

staff_df

,ID_Staff,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [121]:
#select relevant columns
staff_df.drop(columns = ['ID_Staff'], inplace=True)
staff_df

,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [122]:
#format date columns
staff_df['BirthDate__c'] = pd.to_datetime(staff_df['BirthDate__c']).dt.date

staff_df

,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [123]:
#convert to dictionary
staff_data_records = staff_df.to_dict(orient='records')
staff_data_records

[{'EmployeeID__c': '000184220',
  'LastName__c': 'Luongo',
  'FirstName__c': 'Darick',
  'MiddleName__c': 'Nico',
  'BirthDate__c': datetime.date(1995, 7, 5)},
 {'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': datetime.date(1992, 4, 22)},
 {'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': datetime.date(1993, 5, 21)},
 {'EmployeeID__c': '000159108',
  'LastName__c': 'Rodney',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Nicole',
  'BirthDate__c': datetime.date(1994, 6, 7)}]

### Insert `Staff` records into SalesForce

In [124]:
for rec in staff_data_records:
 
    record = {
        'EmployeeID__c': rec['EmployeeID__c'],
        'LastName__c': rec['LastName__c'],
        'FirstName__c': rec['FirstName__c'],
        'MiddleName__c': rec['MiddleName__c'],
        'BirthDate__c': str(rec['BirthDate__c'])
    }
        
    try:
        sf.Staff__C.create(record)
    except Exception as e:
        print(e)

## Pull Staff from SF and join with Staff from AWS to join with Staff Assignment from Salesforce 
This is to get the original `Staff` ID column and to join with the SalesForce `Staff` "Name"

In [128]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT EmployeeID__c, Name FROM staff__c")
for row in data:
    rec = {
        'ID_staff__c': row['Name'], # this is a critical line of code
        'EmployeeID__c': row['EmployeeID__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'ID_staff__c': 'a034x000002jjzJ', 'EmployeeID__c': '000159108'},
 {'ID_staff__c': 'a034x000002jjzE', 'EmployeeID__c': '000160655'},
 {'ID_staff__c': 'a034x000002jjz9', 'EmployeeID__c': '000130109'},
 {'ID_staff__c': 'a034x000002jjz4', 'EmployeeID__c': '000184220'}]

In [129]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_staff__c,EmployeeID__c
0,a034x000002jjzJ,000159108
1,a034x000002jjzE,000160655
2,a034x000002jjz9,000130109
3,a034x000002jjz4,000184220


In [133]:
# Staff DF from AWS MySQL
staff_aws = pd.read_sql("SELECT EmployeeID, ID_Staff FROM staff", conn)
staff_aws

,EmployeeID,ID_Staff
0,000184220,1
1,000130109,2
2,000160655,3
3,000159108,4


In [134]:
staff_aws.rename(columns = {
    'EmployeeID':'EmployeeID__c'
}, inplace = True)

staff_aws

,EmployeeID__c,ID_Staff
0,000184220,1
1,000130109,2
2,000160655,3
3,000159108,4


In [143]:
staff_junction = pd.merge(staff_lookup_df, staff_aws, how = 'left')
staff_junction = staff_junction[['ID_staff__c', 'ID_Staff']]

,ID_staff__c,ID_Staff
0,a034x000002jjzJ,4
1,a034x000002jjzE,3
2,a034x000002jjz9,2
3,a034x000002jjz4,1


# Join `StaffAssignment` DataFrame with `Class` lookup table and `Staff` junction table

In [141]:
#merge with Class look up to get the `Name` column added
staffassignment_df = pd.merge(staffassignment_df, class_lookup_df, how='left')
staffassignment_df

,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c,ID_Class__c,Section__c,Start_Date__c
0,1,1,Teacher Assistant,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
1,4,1,Teacher Assistant,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16
2,3,1,Instructor,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16


In [144]:
#merge with Staff junction to get the `Name` column added
staffassignment_df = pd.merge(staffassignment_df, staff_junction, how='left')
staffassignment_df

,ID_Staff,ID_Class,Role__c,StartDate__c,EndDate__c,ID_Class__c,Section__c,Start_Date__c,ID_staff__c
0,1,1,Teacher Assistant,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a034x000002jjz4
1,4,1,Teacher Assistant,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a034x000002jjzJ
2,3,1,Instructor,2020-09-16,None,a014x000008VFlQ,GWU-ARL-DATA-PT-09-0,2020-09-16,a034x000002jjzE


In [145]:
staffassignment_df = staffassignment_df[['ID_Class__c', 'ID_staff__c', 'Role__c', 'StartDate__c', 'EndDate__c']]
staffassignment_df

,ID_Class__c,ID_staff__c,Role__c,StartDate__c,EndDate__c
0,a014x000008VFlQ,a034x000002jjz4,Teacher Assistant,2020-09-16,None
1,a014x000008VFlQ,a034x000002jjzJ,Teacher Assistant,2020-09-16,None
2,a014x000008VFlQ,a034x000002jjzE,Instructor,2020-09-16,None


In [156]:
#convert to dictionary
staffassignment_records = staffassignment_df.to_dict('records')
staffassignment_records

[{'ID_Class__c': 'a014x000008VFlQ',
  'ID_staff__c': 'a034x000002jjz4',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008VFlQ',
  'ID_staff__c': 'a034x000002jjzJ',
  'Role__c': 'Teacher Assistant',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008VFlQ',
  'ID_staff__c': 'a034x000002jjzE',
  'Role__c': 'Instructor',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None}]

## Insert `StaffAssignment` Records into SalesForce

In [158]:
#load into salesforce
for rec in staffassignment_records:

    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_staff__c': rec['ID_staff__c'],
        'Role__c': rec['Role__c'],
        'EndDate__c': rec['EndDate__c'],
        'StartDate__c': str(rec['StartDate__c'])
    }
    
    try:
        sf.Staff_Assignment__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
sf.bulk.Staff_Assignment__c.insert(staffassignment_records)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [149]:
staffassignment_records = sf.query("SELECT Id FROM Staff_Assignment__c")
recs_to_delete = [{'Id': r['Id']} for r in staffassignment_records['records']]
recs_to_delete

[{'Id': 'a024x000002hKgCAAU'},
 {'Id': 'a024x000002hKgHAAU'},
 {'Id': 'a024x000002hKgMAAU'}]

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [150]:
for rec in recs_to_delete:
    try:
        sf.Staff_Assignment__c.delete(rec['Id'])
    except Exception as e:
        print(e)